<a href="https://colab.research.google.com/github/angelakorm/emotion-recogniton/blob/main/Emotion_Recognition_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

In [85]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import kagglehub
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score

In [86]:
import os
os.environ["WANDB_DISABLED"] = "true"

## Import dataset

In [87]:
path = kagglehub.dataset_download("rahmasleam/emotions-rml-dataset")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/emotions-rml-dataset


In [88]:
dataset = pd.read_csv(path + '/Emotion_classify_Data.csv')

In [89]:
X = dataset['Comment'].astype(str).tolist()
y = dataset.iloc[:, -1]

In [90]:
le = LabelEncoder()
y = le.fit_transform(y)

## Split dataset into train and test set

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Tokenization with BERT

In [92]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [93]:
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=128)

## Prepare dataset for BERT

In [94]:
class CommentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.encodings['input_ids'][idx]),
            'attention_mask': torch.tensor(self.encodings['attention_mask'][idx]),
            'labels': torch.tensor(self.labels[idx])
        }

    def __len__(self):
        return len(self.labels)

train_dataset = CommentDataset(train_encodings, y_train)
test_dataset = CommentDataset(test_encodings, y_test)

## Load model and train arguments

In [95]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [96]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_strategy="no",
    logging_dir='./logs',
    logging_steps=10,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


## Trainer

In [97]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

Step,Training Loss
10,1.071700
20,0.984400
30,0.813900
40,0.608400
50,0.801100
60,0.523400
70,0.445700
80,0.428100
90,0.294800
100,0.233100


TrainOutput(global_step=594, training_loss=0.2030780010226399, metrics={'train_runtime': 141.843, 'train_samples_per_second': 66.961, 'train_steps_per_second': 4.188, 'total_flos': 424643472690444.0, 'train_loss': 0.2030780010226399, 'epoch': 2.0})

## Predict

In [98]:
preds_output = trainer.predict(test_dataset)
preds = np.argmax(preds_output.predictions, axis=1)

## Evaluation

In [99]:
cm = confusion_matrix(y_test, preds)
print(cm)
acc = accuracy_score(y_test, preds)
print(acc)

[[389  23   4]
 [  6 367   0]
 [  2   4 393]]
0.9671717171717171
